In [1]:
#r "nuget: Microsoft.SemanticKernel, *-*"

Installed Packages Microsoft.SemanticKernel, 0.17.230704.3-preview

In [2]:
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.SemanticFunctions;
using Microsoft.SemanticKernel.SkillDefinition;
using Microsoft.SemanticKernel.TemplateEngine;
using System.Collections.Generic;
using System.Threading.Tasks;

In [3]:

IKernel kernel = Kernel.Builder.WithAzureTextCompletionService("GPT3Model", "Your AOAI API Endpoint", "Your AOAI API Key").Build();
            

In [4]:
string skPrompt = """

请帮我把 {{$input}} 进行分类，类型包括天气，课程，生成式，如果不清楚，请回答不确认，参考如下：

问: 会下雨吗？ 类别:天气
问: 今天温度？ 类别:天气
问: 适度多少？ 类别:天气
问: 什么是新能源车？ 类别: 课程
问: 电动车的特点 类别: 课程
问: 概念是什么？ 类别: 课程
问: 课程相关的内容有哪些？ 类别: 课程
问: 写一首诗歌？ 类别: 生成式
问: 翻译一下 类别: 生成式
问: 计算结果 类别: 生成式

天气输出 1 ， 课程输出 2 ， 生成式输出 3 ，不确认输出 0，并参考以下格式输出

{""question"":""{{$input}}"",""label"":""{{$label}}""}

""";

In [5]:
var promptConfig = new PromptTemplateConfig
{
    Completion =
    {
      MaxTokens = 60,
      FrequencyPenalty = (float)0,
      PresencePenalty = (float)0
    }
};

In [6]:
var promptTemplate = new PromptTemplate(
    skPrompt,                        // Prompt template defined in natural language
    promptConfig,                    // Prompt configuration
    kernel                           // SK instance
);

In [7]:
skPrompt


请帮我把 {{$input}} 进行分类，类型包括天气，课程，生成式，如果不清楚，请回答不确认，参考如下：

问: 会下雨吗？ 类别:天气
问: 今天温度？ 类别:天气
问: 适度多少？ 类别:天气
问: 什么是新能源车？ 类别: 课程
问: 电动车的特点 类别: 课程
问: 概念是什么？ 类别: 课程
问: 课程相关的内容有哪些？ 类别: 课程
问: 写一首诗歌？ 类别: 生成式
问: 翻译一下 类别: 生成式
问: 计算结果 类别: 生成式

天气输出 1 ， 课程输出 2 ， 生成式输出 3 ，不确认输出 0，并参考以下格式输出

{""question"":""{{$input}}"",""label"":""{{$label}}""}


In [8]:

var functionConfig = new SemanticFunctionConfig(promptConfig, promptTemplate);

In [9]:
var classificationFunction = kernel.RegisterSemanticFunction("QASkill", "QuestionClassification", functionConfig);

In [10]:
classificationFunction


{"Name":"QuestionClassification","SkillName":"QASkill","Description":"","IsSemantic":true,"IsSensitive":false,"TrustServiceInstance":{},"RequestSettings":{"Temperature":0,"TopP":0,"PresencePenalty":0,"FrequencyPenalty":0,"MaxTokens":60,"StopSequences":[],"ResultsPerPrompt":1,"ChatSystemPrompt":"Assi... Name QuestionClassification SkillName QASkill Description IsSemantic True IsSensitive False TrustServiceInstance Microsoft.SemanticKernel.Security.TrustService RequestSettings Microsoft.SemanticKernel.AI.TextCompletion.CompleteRequestSettings Temperature 0 TopP 0 PresencePenalty 0 FrequencyPenalty 0 MaxTokens 60 StopSequences ResultsPerPrompt 1 ChatSystemPrompt Assistant is a large language model. TokenSelectionBiases (empty) Parameters index value 0 Microsoft.SemanticKernel.SkillDefinition.ParameterView Name input Description DefaultValue 1 Microsoft.SemanticKernel.SkillDefinition.ParameterView Name label Description DefaultValue

In [11]:
var input = "今天广州天气怎么样？";

In [12]:
var classification = await kernel.RunAsync(input, classificationFunction);

Console.WriteLine(classification);


答案：{"question":"今天广州天气怎么样？","label":"1"}
